In [3]:
import oineus as oin
import numpy as np
import torch
import diode

def get_points(file):
    points = []
    with open(file) as f:
        for (i, line) in enumerate(f):
            if i >= 3:
                x, y, z = line.split()
                points.append(np.array([float(x), float(y), float(z)]))
    return points

def get_interface_diagram_and_filtration(points, n_atoms_per_mol):
    points = np.asarray(points)
    simplices = diode.fill_alpha_shapes(points)
    fil = oin.Filtration_double([oin.Simplex_double(s[0], s[1]) for s in simplices])

    def is_multi(sigma):
        return len(set(v // n_atoms_per_mol for v in sigma.vertices)) >= 2
    fil = fil.subfiltration(is_multi)

    def recalculate_filtration_value(cell):
        parts = [v // n_atoms_per_mol for v in cell.vertices]
        n_parts = len(set(parts))
        p_agg = [np.array([0.0, 0.0, 0.0]) for _ in range(n_parts)]
        weights = [0 for _ in range(n_parts)]
        for i, p in enumerate(parts):
            p_agg[p] += points[cell.vertices[i]]
            weights[p] += 1
        bcs = [p_agg[i] / weights[i] for i in range(n_parts)]
        filtration_value = sum([np.linalg.norm(bcs[i] - bcs[j]) for i in range(n_parts) for j in range(i + 1, n_parts)]) / n_parts
        # convert to native Python float from numpy
        return float(filtration_value)

    new_values = [recalculate_filtration_value(cell) for cell in fil]
    fil.set_values(new_values, 1)

    dcmp = oin.Decomposition(fil, True)
    params = oin.ReductionParams()
    params.clearing_opt = False
    dcmp.reduce(params)
    dgm = dcmp.diagram(fil, include_inf_points=False)
    return dgm, fil

points = get_points("test.txt")
get_interface_diagram_and_filtration(points, 1206)

(<oineus._oineus.Diagrams_double at 0x135c43eb0>,
 Filtration(size = 1681, 
 cells = [([1205, 1206], 0.613846),
 ([1205, 1209], 0.663538),
 ([1204, 1207], 0.764114),
 ([1204, 1206], 1.19718),
 ([1204, 1209], 1.21024),
 ([1204, 1208], 1.21415),
 ([1205, 1210], 1.21716),
 ([1203, 1207], 1.22789),
 ([1205, 1207], 1.24659),
 ([1203, 2405], 1.33376),
 ([1203, 1206], 1.35788),
 ([1205, 1208], 1.41953),
 ([1203, 1208], 1.48735),
 ([1205, 1211], 1.61389),
 ([1204, 1662], 1.67163),
 ([1205, 2398], 1.74874),
 ([1205, 1212], 1.76828),
 ([1204, 2400], 1.77998),
 ([1205, 2400], 1.80148),
 ([1203, 2399], 1.81227),
 ([1203, 2401], 1.8297),
 ([1204, 2401], 1.84798),
 ([1203, 2397], 1.89974),
 ([1203, 2400], 1.94965),
 ([1204, 2398], 1.95328),
 ([1203, 2395], 2.02028),
 ([1205, 2395], 2.02379),
 ([1203, 2398], 2.04534),
 ([1203, 2402], 2.06178),
 ([1204, 1669], 2.51878),
 ([1204, 1667], 2.55961),
 ([1203, 1669], 2.68175),
 ([1203, 1671], 3.5988),
 ([1203, 1672], 3.6931),
 ([515, 1399], 3.76917),
 ([515

IndexError: unordered_map::at: key not found